# Zero-shot perplexity tuning

In [ ]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "src").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
PROJECT_ROOT
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

# DATA_LIMIT = 10000  # set to None to use the full cached RAID split
DATA_LIMIT = None  # set to None to use the full cached RAID split
SAMPLE_SEED = 42
# TEST_RATIO = 0.2
TEST_RATIO = 0.02
SHOW_PROGRESS = True


In [ ]:
dataset = data.load_raid(limit=DATA_LIMIT, sample_seed=SAMPLE_SEED, test_ratio=TEST_RATIO)
test_ds = dataset["test"]
texts = test_ds[config.TEXT_FIELD]
labels = np.array(test_ds[config.LABEL_FIELD])

train_dataset = dataset["train"]

print(f"Test samples: {len(texts)}")

Available prompt models: ('heuristic', 'llama-3.2-1b-instruct')
Test samples: 9360


## Threshold sweep on training split

In [5]:
from src.models.perplexity_detector import PerplexityDetector

ppl_detector = PerplexityDetector(show_progress=True)
train_texts = train_dataset[config.TEXT_FIELD]
print(f'Calibrating on {len(train_texts)} training samples')

Calibrating on 458625 training samples


In [ ]:
train_stats = ppl_detector.score(train_texts)
train_nll = train_stats.nll
finite_mask = np.isfinite(train_nll)
finite_nll = train_nll[finite_mask]
print(f'Finite NLL count: {finite_mask.sum()} / {len(train_nll)}')

/home/erikhbj/Documents/tdt13/TDT13_AI_text_prediction/.venv/lib/python3.11/site-packages/rich/live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


In [ ]:
percentiles = [0.5, 1.0, 2.0, 5.0, 10.0, 25.0, 50.0]  # human share is ~2.2%
for p in percentiles:
    thresh = np.percentile(finite_nll, p)
    print(f'Percentile {p:>4}: threshold={thresh:.4f}')